## Final project
1. Build a neural network model for a very simple classification task on the data provided here.
2. We are using Tensor flow lib.

## Part 1: Find the simplest neural networks

### Import required libs.

In [1]:
import tensorflow as tf
import numpy as np
import matplotlib.pyplot as plt
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.metrics import confusion_matrix
import itertools

### Read Input data

In [2]:
#read data from file(data set)
Data = pd.read_csv('input_data.csv') 
#separate the output attribute as label for training puropse
output = Data['Label']

### Convert label into numerical format

##### our output label is in the form of char (r,g,b) but in arctitecture we can use numeric valuse so we create label in interger form as
    r = 0
    g = 1
    b = 2

In [3]:
label= []
for value in output:
    if value=='r':
        label.append(0)
    elif value=='g':
        label.append(1)
    elif value=='b':
        label.append(2)
label = np.array(label)        
data = Data.drop('Label',axis=1)

##### spliting the data set 80% for training the model and 20% for the testing and evaluating the trained model

In [4]:
trainingData, testingData, trainingLabel, testingLabel = train_test_split(data, label, test_size=0.2)
print(trainingLabel.shape)

(548,)


#### Multilayer architecure 
1. Total 4 layers use in first layer 64 neurons, in 2nd layer 128, in 3rd 256 and in last(output) layer 3 Neurons used.
2. Used relu as activation function in all layers except last layer. In last layer use softmax as activation function
3. Sparse softmax cross entropy is used for loss calculation
4. Adam optimizer is used with learning rate of 0.0001 for avoiding overfiting and better learning  use dropout(0.5).

In [5]:
epochs=40
model = tf.keras.Sequential();
model.add(tf.keras.layers.Dense(units=64, activation= tf.nn.relu, input_dim=2))
model.add(tf.keras.layers.Dropout(0.5))
model.add(tf.keras.layers.Dense(units=128, activation= tf.nn.relu))
model.add(tf.keras.layers.Dropout(0.5))
model.add(tf.keras.layers.Dense(units=256, activation= tf.nn.relu))
#model.add(tf.keras.layers.Dropout(0.5))
model.add(tf.keras.layers.Dense(units=3, activation= tf.nn.softmax))
model.compile(loss=tf.losses.sparse_softmax_cross_entropy, optimizer=tf.keras.optimizers.Adam(0.0001), metrics=['accuracy'])
history = model.fit(trainingData,trainingLabel, validation_data = (testingData, testingLabel), epochs=epochs)

Instructions for updating:
Colocations handled automatically by placer.
Instructions for updating:
Please use `rate` instead of `keep_prob`. Rate should be set to `rate = 1 - keep_prob`.
Instructions for updating:
Use tf.cast instead.
Train on 548 samples, validate on 137 samples
Epoch 1/40
548/548 [==============================] - 1s 2ms/sample - loss: 0.8646 - acc: 0.7664 - val_loss: 0.7076 - val_acc: 1.0000
Epoch 2/40
548/548 [==============================] - 0s 179us/sample - loss: 0.7062 - acc: 0.9690 - val_loss: 0.6945 - val_acc: 1.0000
Epoch 3/40
548/548 [==============================] - 0s 170us/sample - loss: 0.6830 - acc: 0.9982 - val_loss: 0.6935 - val_acc: 1.0000
Epoch 4/40
548/548 [==============================] - 0s 177us/sample - loss: 0.6846 - acc: 0.9945 - val_loss: 0.6934 - val_acc: 1.0000
Epoch 5/40
548/548 [==============================] - 0s 174us/sample - loss: 0.6789 - acc: 1.0000 - val_loss: 0.6931 - val_acc: 1.0000
Epoch 6/40
548/548 [=====================

In [6]:
acc = history.history['acc']
val_acc = history.history['val_acc']

loss = history.history['loss']
val_loss = history.history['val_loss']

epochs_range = range(epochs)


### Evaluate Model

In [7]:
evlauteModel = model.evaluate(testingData, testingLabel)

137/137 [==============================] - 0s 57us/sample - loss: 0.6847 - acc: 0.9635


### Pleaes note: As you increae layers and number of Neurons used in each layers we get better accuracy ; in this case it's 1

### Plot graphs

In [8]:
# Commented for testing ; uncomment section for final execution
"""plt.figure(figsize=(6,6))
plt.subplot(1, 2, 1)
plt.plot(epochs_range, acc, label='Training Accuracy')
plt.plot(epochs_range, val_acc, label='Validation Accuracy')
plt.legend(loc='lower right')
plt.title('Training and Validation Accuracy')

plt.subplot(1, 2, 2)
plt.plot(epochs_range, loss, label='Training Loss')
plt.plot(epochs_range, val_loss, label='Validation Loss')
plt.legend(loc='upper right')
plt.title('Training and Validation Loss')
plt.savefig('./foo.png')
plt.show()"""

"plt.figure(figsize=(6,6))\nplt.subplot(1, 2, 1)\nplt.plot(epochs_range, acc, label='Training Accuracy')\nplt.plot(epochs_range, val_acc, label='Validation Accuracy')\nplt.legend(loc='lower right')\nplt.title('Training and Validation Accuracy')\n\nplt.subplot(1, 2, 2)\nplt.plot(epochs_range, loss, label='Training Loss')\nplt.plot(epochs_range, val_loss, label='Validation Loss')\nplt.legend(loc='upper right')\nplt.title('Training and Validation Loss')\nplt.savefig('./foo.png')\nplt.show()"

In [9]:
classes = 3
prediction = model.predict(testingData)

prediction = np.argmax(prediction, axis=1)
cm = confusion_matrix(testingLabel,prediction)



def plot_confusion_matrix(cm, classes,
                          normalize=False,
                          title='Confusion matrix',
                          cmap=plt.cm.Blues):
    """
    This function prints and plots the confusion matrix.
    Normalization can be applied by setting `normalize=True`.
    """
    if normalize:
        cm = cm.astype('float') / cm.sum(axis=1)[:, np.newaxis]
        print("Normalized confusion matrix")
    else:
        print('Confusion matrix, without normalization')

    print(cm)
    # Commented for testing ; uncomment section for final execution
    """plt.imshow(cm, interpolation='nearest', cmap=cmap)
    plt.title(title)
    plt.colorbar()
    tick_marks = np.arange(len(classes))
    plt.xticks(tick_marks, classes, rotation=45)
    plt.yticks(tick_marks, classes)

    fmt = '.2f' if normalize else 'd'
    thresh = cm.max() / 2.
    for i, j in itertools.product(range(cm.shape[0]), range(cm.shape[1])):
        plt.text(j, i, format(cm[i, j], fmt),
                 horizontalalignment="center",
                 color="white" if cm[i, j] > thresh else "black")

    plt.ylabel('True label')
    plt.xlabel('Predicted label')
    plt.tight_layout()"""

### Build confusion  metrix

In [10]:
cm_plot_labels = ["R","G","B"]
plot_confusion_matrix(cm, cm_plot_labels, title='Confusion Matrix')

Confusion matrix, without normalization
[[115   0   0]
 [ 12   0   0]
 [  5   0   5]]


## Part 2: Build new attributes

In [11]:
X1 = Data['X1'] # extracting first col from data set
X2 = Data['X2'] # extracting second col from data set
X3 = []
X4 = []
X5 = []

In [12]:
for i in range(len(X1)):
    X3.append(X1[i]*X2[i])
    X4.append(X2[i])
    X5.append(X1[i]*X2[i])

In [13]:
newDataSet = Data[['X1','X2']]
newDataSet['X3'] = X3
newDataSet['X4'] = X4
newDataSet['X5'] = X5

### Output

In [14]:
newDataSet

,X1,X2,X3,X4,X5
0,16.263398,13.299206,216.290282,13.299206,216.290282
1,0.775408,23.986692,18.599466,23.986692,18.599466
2,29.170503,-3.287474,-95.897279,-3.287474,-95.897279
3,6.739044,-28.033329,-188.917824,-28.033329,-188.917824
4,3.216100,22.013695,70.798239,22.013695,70.798239
5,47.374906,7.925541,375.471747,7.925541,375.471747
6,24.064604,14.726109,354.377985,14.726109,354.377985
7,12.667897,-16.456068,-208.463776,-16.456068,-208.463776
8,4.194880,20.421886,85.667357,20.421886,85.667357
9,48.451924,5.783617,280.227379,5.783617,280.227379
